In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import re

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('Data\Conf2DataLSTM.txt')
data.columns = ['cumle','sinif']
data = data[['cumle','sinif']]

In [3]:
# Veri setimizin işlenebilmesi için text verileri numaralara çevirmemiz gerekir
# Keras bu işlem için hazır bir mekanizma sunmaktadır
# Tokenizer sınıfı data içerisinde verilen cümleleri analiz ederek kelimelerin sıklıklarını hesaplar
# Parameter: num_words = En sık geçen 25000 kelimeye odaklan, diğerleri önemli değil demek
tokenizer = Tokenizer(split=' ',num_words=25000)
# Her bir kelimenin sıklığını(frekansını) hesaplar
tokenizer.fit_on_texts(data['cumle'].values)
# Tüm cümleler tam sayı dizisine dönüştürülür
X = tokenizer.texts_to_sequences(data['cumle'].values)
# Bütün metinlerimiz 400 sütundan oluşan bir dizi ile temsil edilecek
# Çok kısa metinler 0'lar ile doldurulacak, çok uzun metinler ise kesilecek
X = pad_sequences(X,maxlen=400)

In [4]:
# Yapay sinir ağı modelimiz
embed_dim = 128
lstm_out = 128
def build_model():
    model = Sequential()
    # Her bir kelimenin temsil edileceği vektör boyutu. Bu örnek için her bir kelime 128 boyutunda
    # bir vektör ile temsil edilir.
    model.add(Embedding(25000, embed_dim,input_length = X.shape[1], dropout=0.2))
    model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

  # Çıktılarımızı kategorik hale getirdik (Opsiyonel)
Y = pd.get_dummies(data['sinif']).values
Y[0]

array([1, 0, 0, 0], dtype=uint8)

In [5]:
# Verinin %90'i train, %10'si test verisi olacak şekilde ayrılacak
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 1)

model = build_model()
# Oluşturulan model train verileri ile eğitilir
# Burada yapay sinir ağını eğitmeye başlıyoruz
# nb_epoch: İterasyon sayısı
model.fit(X_train, Y_train, nb_epoch = 5, batch_size=32, verbose = 2)
# Train verileri ile model eğitildikten sonra test dataları ile doğruluk oranlarına bakılır.
score = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score[1]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/5
 - 32s - loss: 1.1436 - acc: 0.5363
Epoch 2/5
 - 34s - loss: 0.3531 - acc: 0.9014
Epoch 3/5
 - 34s - loss: 0.1089 - acc: 0.9714
Epoch 4/5
 - 34s - loss: 0.0558 - acc: 0.9869
Epoch 5/5
 - 34s - loss: 0.0256 - acc: 0.9942
score: 0.93


In [14]:
# Çıktılarımız => 0: Kızmak, 1: Korkmak, 2: Mutluluk, 3:Üzüntü
outputs = ["Kızmak", "Korkmak", "Mutluluk", "Üzüntü"]

# Verilen örnekler Tokenizer yapısı ile tam sayı dizisine dönüştürülür
# Daha sonra eğitilen modele sırayla verilerek anlam analizi sonuçları elde edilir
# Her Cümlenin yüzde kaç olumlu ve olumsuz olduğuna dair bilgiler çıktı olarak verilir

my_text = ["çok kız ne kadar ayıp şey"]

sequences = tokenizer.texts_to_sequences(my_text)
data = pad_sequences(sequences, maxlen=400)
predictions = model.predict(data)
print(predictions[0])
predictions = list(predictions)
print(outputs[predictions.index(max(predictions))])

[0.7602026  0.03576458 0.05444506 0.14958785]
Kızmak


In [0]:
# Burada örnek olarak test edilen cümle, üzerinde doğal dil işleme(NLP) işlemleri uygulanmış bir veridir
# Programın amacı analiz olduğundan NLP işlemleri buraya eklenmedi
# Eğer yapılmak istenilirse bütün kelimelerin kök halleri alınır